그래프를 만들자 

Model, sampling_way, target_sr, acc, recall, precision, f1, auc_score

1. 모든 모델을 그래프 하나에 담자
    - 각 기법별로 그리면 됨.
    - 총 기법은 17가지가 있음.
    - 그래프 한 figure에는 5개의 그래프가 
        - 각각 f1, auc, recall, precision, accuracy를 담고 있어야함.
    - 이렇게 총 17개의 figure를 뽑아야함.

'abs_max(our_custom)', 'kaiser_best', 'kaiser_fast', 'fft', 'scipy', 'polyphase', 'linear', 'zero_order_hold', 'sinc_best', 'sinc_medium', 'sinc_fastest', 'soxr_vhq', 'soxr_hq', 'soxr_mq', 'soxr_lq', 'soxr_hq', 'soxr_qq'

In [17]:
resampling_way = ['abs_max(our_custom)', 'kaiser_best', 'kaiser_fast', 'fft', 'scipy', 'polyphase', 'linear', 'zero_order_hold', 'sinc_best', 'sinc_medium', 'sinc_fastest', 'soxr_vhq', 'soxr_hq', 'soxr_mq', 'soxr_lq', 'soxr_hq', 'soxr_qq']
len(resampling_way)

17

In [18]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import glob
import matplotlib.patches as mpatches
import matplotlib.lines as mlines

In [19]:
fan_target_id_00 = pd.read_csv('./data/fan_target_sr_id00.csv')
fan_target_id_02 = pd.read_csv('./data/fan_target_sr_id02.csv')
fan_target_id_04 = pd.read_csv('./data/fan_target_sr_id04.csv')
fan_target_id_06 = pd.read_csv('./data/fan_target_sr_id06.csv')

In [20]:
fan_target_id_00['Model'] = 'id_00'
fan_target_id_02['Model'] = 'id_02'
fan_target_id_04['Model'] = 'id_04'
fan_target_id_06['Model'] = 'id_06'
data = pd.concat([fan_target_id_00,fan_target_id_02,fan_target_id_04,fan_target_id_06])

In [21]:
## input : 
resample = 'kaiser_best'
data

,target_sr,acc,recall,precision,f1,auc_score,Model
0,40,0.717,0.201,0.516,0.289,0.589,id_00
1,50,0.972,0.910,0.991,0.949,0.996,id_00
2,64,0.995,0.984,1.000,0.992,1.000,id_00
3,80,0.985,0.947,1.000,0.973,1.000,id_00
4,100,0.979,0.930,0.996,0.962,0.999,id_00
...,...,...,...,...,...,...,...
18,3200,0.937,0.765,0.994,0.865,0.997,id_06
19,4000,0.932,0.747,0.994,0.853,0.998,id_06
20,8000,0.930,0.737,0.994,0.847,0.999,id_06
21,10000,0.929,0.728,1.000,0.843,0.999,id_06


## Point Plot

In [22]:






def whole_best_vis(data, resample:str):
    '''
    모든 모델링 결과들을 시각화하는 함수
        input : - data : 데이터프레임
                - resample : resampling type
        output : 가장 수치가 높았던 값들 데이터프레임 2개
    
    '''

    ID_li = ['id_00', 'id_02', 'id_04', 'id_06']
    color_li = ['r', 'y', 'g', 'b']
    factor = ['acc', 'recall', 'precision', 'auc_score', 'f1']


    fig = plt.figure(figsize=(30,20), dpi=300)
    axs = fig.subplots(3,2)

    # date = data[data['sampling_way'] == 'kaiser_best']
    val_dic = {}
    ind_dic = {}
    for ind, fac in enumerate(factor):
        val_dic[fac] = []
        ind_dic[fac] = []
        for ID, color in zip(ID_li, color_li):
            con_id_ = data['Model'] == ID

            X1 = range(len(data[con_id_]['target_sr'].values))
            Y = data[con_id_]


            maxval = np.argmax(Y[fac])
            val_dic[fac].append(Y[fac][maxval])
            ind_dic[fac].append(data[con_id_]['target_sr'][maxval])

            row = ind // 2 
            col = ind % 2 

            sns.pointplot(x="target_sr", y=fac,data=Y, color=color, label='{}'.format(ID), ax=axs[row, col])
#             _=axs[row, col].plot(X1,Y, color=color ,label='{}'.format(ID))
            _=axs[row, col].axvline(maxval, color=color, linestyle='--', linewidth=2)
            _=axs[row, col].set_title('{}'.format(fac), fontsize=20)
            _=axs[row, col].set_xticks(X1)
            _=axs[row, col].set_xticklabels(data[con_id_]['target_sr'], fontsize=20, rotation=45)
            



        # 범례 지정하기
        # 박스형태 범례
        variable_x = mpatches.Patch(color='r',label='id_00')
        variable_y = mpatches.Patch(color='y',label='id_02')
        variable_z = mpatches.Patch(color='g',label='id_04')
        variable_w = mpatches.Patch(color='b',label='id_06')


        #범례 나타내기
        _=axs[row, col].legend(handles=[variable_x, variable_y, variable_z,variable_w], loc='best', fontsize=10)
#         _=axs[row, col].legend(loc='best', fontsize=10)
        _=axs[row, col].set_ylim(0,1.2)


    X = np.arange(len(val_dic['acc']))

    for ind, fac in enumerate(factor):
        axs[2,1].bar(X+(ind*0.1), val_dic[fac], width=0.15, align='center', label='{}'.format(fac))

        for ii, index in enumerate(ind_dic[fac]):
            x_pos = X[ii] + (ind * 0.1)
            y_pos = val_dic[fac][ii]
            axs[2,1].text(x_pos, y_pos, '{}'.format(index), ha='left', va='bottom', fontsize=15)

    _= axs[2,1].set_xticks(X+0.2)
    _= axs[2,1].set_xticklabels(['id_00', 'id_02', 'id_04', 'id_06'], fontsize=20)
    _= axs[2,1].legend(loc='best', fontsize=20)
    _= axs[2,1].set_title('Every Max Values', fontsize=20)


    _=fig.suptitle('{}'.format(resample), fontsize=30)

    fig.tight_layout()

    val_df = pd.DataFrame(val_dic, index = ID_li)
    ind_df = pd.DataFrame(ind_dic, index = ID_li)

    return val_df, ind_df

In [23]:
aa = whole_best_vis(data, resample)

In [24]:
aa[1]

,acc,recall,precision,auc_score,f1
id_00,64,64,64,64,64
id_02,125,125,1000,10000,125
id_04,125,125,10000,125,125
id_06,1600,1600,1600,1600,1600
